## 1. Setup & Imports

In [ ]:
%pip install transformers datasets torch scikit-learn -q

In [ ]:
import pandas as pd
import numpy as np
import torch
import os
import gc
import sys
from pathlib import Path
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Add parent directories to path for imports
sys.path.insert(0, str(Path.cwd().parent.parent))
sys.path.insert(0, str(Path.cwd().parent))

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 2. Configuration

Set your data paths and training parameters here.

In [ ]:
# ============================================
# DATA CONFIGURATION
# ============================================

# Path to preprocessed training data (output from preprocessing pipeline)
TRAIN_DATA_PATH = '../processed/train_arb_bas.csv'

# Path to preprocessed dev data
DEV_DATA_PATH = '../processed/dev_arb_bas.csv'

# Column names from preprocessing pipeline
TEXT_COL = 'text_normalized'  # Use dialect-normalized text for training
LABEL_COL = 'polarization'
TOXIC_RATIO_COL = 'toxic_ratio'

# ============================================
# CASCADING CLASSIFIER CONFIGURATION
# ============================================

# Enable cascading classifier for pre-filtering
USE_CASCADING = True

# Thresholds for rule-based classification
TOXIC_THRESHOLD = 0.15  # Ratio above this -> TOXIC (confidence 0.9)
CLEAN_THRESHOLD = 0.02  # Ratio below this -> CLEAN (confidence 0.85)

# ============================================
# TRAINING CONFIGURATION
# ============================================

MODEL_NAME = "UBC-NLP/MARBERTv2"
RANDOM_SEED = 42
N_FOLDS = 8
MAX_LENGTH = 128

training_config = {
    'num_train_epochs': 4,
    'per_device_train_batch_size': 16,
    'per_device_eval_batch_size': 32,
    'learning_rate': 2e-5,
    'weight_decay': 0.01,
    'warmup_steps': 500,
    'logging_steps': 50,
    'save_strategy': 'epoch',
    'fp16': torch.cuda.is_available(),
    'seed': RANDOM_SEED
}

print("Configuration:")
print(f"  Train data: {TRAIN_DATA_PATH}")
print(f"  Dev data: {DEV_DATA_PATH}")
print(f"  Text column: {TEXT_COL}")
print(f"  Use cascading: {USE_CASCADING}")
print(f"  Model: {MODEL_NAME}")
print(f"  Folds: {N_FOLDS}")

## 3. Load Preprocessed Data

In [ ]:
# Load training data
train_df = pd.read_csv(TRAIN_DATA_PATH)

print("="*80)
print("TRAINING DATA")
print("="*80)
print(f"Size: {len(train_df)}")
print(f"Columns: {train_df.columns.tolist()}")
print(f"\nClass distribution:")
print(train_df[LABEL_COL].value_counts())
print(f"\nClass balance:")
print(train_df[LABEL_COL].value_counts(normalize=True))

# Check for required columns
required_cols = ['id', TEXT_COL, LABEL_COL, TOXIC_RATIO_COL]
missing_cols = [col for col in required_cols if col not in train_df.columns]
if missing_cols:
    print(f"\n⚠️ WARNING: Missing columns: {missing_cols}")
    print("Please run the preprocessing pipeline first!")
else:
    print(f"\n✓ All required columns present")

print(f"\nSample data:")
print(train_df[[TEXT_COL, LABEL_COL, TOXIC_RATIO_COL]].head(3))

In [ ]:
# Load dev data
dev_df = pd.read_csv(DEV_DATA_PATH)

print("="*80)
print("DEV DATA")
print("="*80)
print(f"Size: {len(dev_df)}")
print(f"Columns: {dev_df.columns.tolist()}")
print(f"\nSample data:")
print(dev_df[[TEXT_COL, TOXIC_RATIO_COL]].head(3) if TOXIC_RATIO_COL in dev_df.columns else dev_df.head(3))

## 4. Toxic Ratio Analysis

In [ ]:
if TOXIC_RATIO_COL in train_df.columns:
    print("="*80)
    print("TOXIC RATIO ANALYSIS")
    print("="*80)
    
    # Overall statistics
    print(f"\nOverall Statistics:")
    print(train_df[TOXIC_RATIO_COL].describe())
    
    # Statistics by class
    print(f"\nStatistics by Class:")
    for label in sorted(train_df[LABEL_COL].unique()):
        subset = train_df[train_df[LABEL_COL] == label]
        print(f"\nClass {label}:")
        print(f"  Mean toxic ratio: {subset[TOXIC_RATIO_COL].mean():.4f}")
        print(f"  Std toxic ratio: {subset[TOXIC_RATIO_COL].std():.4f}")
        print(f"  Non-zero ratio samples: {(subset[TOXIC_RATIO_COL] > 0).sum()} ({100*(subset[TOXIC_RATIO_COL] > 0).mean():.1f}%)")
    
    # Cascading classifier potential
    if USE_CASCADING:
        print(f"\n{'='*80}")
        print("CASCADING CLASSIFIER POTENTIAL")
        print(f"{'='*80}")
        
        high_toxic = (train_df[TOXIC_RATIO_COL] > TOXIC_THRESHOLD).sum()
        very_clean = (train_df[TOXIC_RATIO_COL] < CLEAN_THRESHOLD).sum()
        uncertain = len(train_df) - high_toxic - very_clean
        
        print(f"\nWith thresholds: TOXIC > {TOXIC_THRESHOLD}, CLEAN < {CLEAN_THRESHOLD}")
        print(f"  High toxic (rule-based TOXIC): {high_toxic} ({100*high_toxic/len(train_df):.1f}%)")
        print(f"  Very clean (rule-based CLEAN): {very_clean} ({100*very_clean/len(train_df):.1f}%)")
        print(f"  Uncertain (needs model): {uncertain} ({100*uncertain/len(train_df):.1f}%)")
        print(f"\n💡 Model only needs to process {100*uncertain/len(train_df):.1f}% of samples!")
        
        # Accuracy of rule-based decisions on training data
        high_toxic_correct = ((train_df[TOXIC_RATIO_COL] > TOXIC_THRESHOLD) & (train_df[LABEL_COL] == 1)).sum()
        very_clean_correct = ((train_df[TOXIC_RATIO_COL] < CLEAN_THRESHOLD) & (train_df[LABEL_COL] == 0)).sum()
        
        print(f"\nRule-based accuracy (on training data):")
        if high_toxic > 0:
            print(f"  High toxic -> TOXIC: {100*high_toxic_correct/high_toxic:.1f}% accurate")
        if very_clean > 0:
            print(f"  Very clean -> CLEAN: {100*very_clean_correct/very_clean:.1f}% accurate")

## 5. Setup Tokenizer

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(f"✓ Tokenizer loaded: {MODEL_NAME}")
print(f"  Vocab size: {tokenizer.vocab_size}")

In [ ]:
# Tokenization function using normalized text
def tokenize_function(examples):
    return tokenizer(
        examples[TEXT_COL],
        padding='max_length',
        truncation=True,
        max_length=MAX_LENGTH
    )

print(f"✓ Tokenization function defined (using column: '{TEXT_COL}')")

## 6. Prepare Dev Dataset

In [ ]:
# Prepare dev dataset
dev_dataset = Dataset.from_pandas(dev_df[[TEXT_COL]])

print("Tokenizing dev data...")
dev_dataset_tokenized = dev_dataset.map(tokenize_function, batched=True)
dev_dataset_tokenized.set_format('torch', columns=['input_ids', 'attention_mask'])

print(f"✓ Dev dataset tokenized: {len(dev_dataset_tokenized)} samples")

## 7. Setup Cross-Validation

In [ ]:
# Setup stratified k-fold
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)

print(f"✓ {N_FOLDS}-Fold Cross-Validation configured")
print(f"  Random Seed: {RANDOM_SEED}")
print(f"  Stratification: Enabled")

## 8. Helper Functions

In [ ]:
def compute_metrics(eval_pred):
    """Compute metrics for evaluation"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    
    return {
        'accuracy': accuracy,
        'f1': f1
    }

def plot_confusion_matrix(y_true, y_pred, title="Confusion Matrix"):
    """Plot confusion matrix"""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

print("✓ Helper functions defined")

## 9. Train 8-Fold Models

Train 8 models with memory-efficient model saving.

In [ ]:
# Storage for model paths and results
model_paths = []
fold_results = []

# Create directory for saved models
os.makedirs('./saved_models_pipeline', exist_ok=True)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Get indices for stratified k-fold
X = train_df[TEXT_COL].values
y = train_df[LABEL_COL].values

print("="*80)
print(f"STARTING {N_FOLDS}-FOLD CROSS-VALIDATION TRAINING")
print("="*80)

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n{'='*80}")
    print(f"FOLD {fold}/{N_FOLDS}")
    print(f"{'='*80}")
    
    # Split data
    fold_train_df = train_df.iloc[train_idx].reset_index(drop=True)
    fold_val_df = train_df.iloc[val_idx].reset_index(drop=True)
    
    print(f"Training samples: {len(fold_train_df)}")
    print(f"Validation samples: {len(fold_val_df)}")
    print(f"Train class dist: {fold_train_df[LABEL_COL].value_counts().to_dict()}")
    print(f"Val class dist: {fold_val_df[LABEL_COL].value_counts().to_dict()}")
    
    # Create datasets using normalized text
    fold_train_dataset = Dataset.from_pandas(fold_train_df[[TEXT_COL, LABEL_COL]])
    fold_val_dataset = Dataset.from_pandas(fold_val_df[[TEXT_COL, LABEL_COL]])
    
    # Tokenize
    fold_train_dataset = fold_train_dataset.map(tokenize_function, batched=True)
    fold_train_dataset = fold_train_dataset.rename_column(LABEL_COL, 'labels')
    fold_train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    
    fold_val_dataset = fold_val_dataset.map(tokenize_function, batched=True)
    fold_val_dataset = fold_val_dataset.rename_column(LABEL_COL, 'labels')
    fold_val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    
    # Load fresh model
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=2
    )
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_pipeline_fold_{fold}',
        num_train_epochs=training_config['num_train_epochs'],
        per_device_train_batch_size=training_config['per_device_train_batch_size'],
        per_device_eval_batch_size=training_config['per_device_eval_batch_size'],
        learning_rate=training_config['learning_rate'],
        weight_decay=training_config['weight_decay'],
        warmup_steps=training_config['warmup_steps'],
        logging_dir=f'./logs_pipeline_fold_{fold}',
        logging_steps=training_config['logging_steps'],
        eval_strategy="epoch",
        save_strategy=training_config['save_strategy'],
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        seed=training_config['seed'],
        fp16=training_config['fp16'],
        report_to='none'
    )
    
    # Create trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=fold_train_dataset,
        eval_dataset=fold_val_dataset,
        compute_metrics=compute_metrics,
        data_collator=data_collator
    )
    
    # Train
    print(f"\nTraining Fold {fold}...")
    trainer.train()
    
    # Evaluate
    print(f"\nEvaluating Fold {fold}...")
    eval_results = trainer.evaluate()
    
    print(f"\nFold {fold} Results:")
    print(f"  Validation Accuracy: {eval_results['eval_accuracy']:.4f}")
    print(f"  Validation F1 Score: {eval_results['eval_f1']:.4f}")
    
    # Save model
    model_save_path = f'./saved_models_pipeline/fold_{fold}_model'
    trainer.save_model(model_save_path)
    print(f"  Model saved to: {model_save_path}")
    
    # Store results
    model_paths.append(model_save_path)
    fold_results.append({
        'fold': fold,
        'accuracy': eval_results['eval_accuracy'],
        'f1': eval_results['eval_f1']
    })
    
    # Detailed report
    predictions = trainer.predict(fold_val_dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    labels = predictions.label_ids
    
    print(f"\nClassification Report (Fold {fold}):")
    print(classification_report(labels, preds, target_names=['Class 0', 'Class 1']))
    
    # Clear GPU memory
    del model, trainer, fold_train_dataset, fold_val_dataset, predictions
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    print(f"  GPU memory cleared")

print("\n" + "="*80)
print(f"{N_FOLDS}-FOLD CROSS-VALIDATION COMPLETE")
print("="*80)

## 10. Cross-Validation Results

In [ ]:
# Results summary
results_df = pd.DataFrame(fold_results)

print("="*80)
print(f"{N_FOLDS}-FOLD CROSS-VALIDATION RESULTS")
print("="*80)
print("\nPer-Fold Results:")
print(results_df.to_string(index=False))

print(f"\n{'='*80}")
print("SUMMARY STATISTICS")
print(f"{'='*80}")
print(f"Mean Accuracy: {results_df['accuracy'].mean():.4f} ± {results_df['accuracy'].std():.4f}")
print(f"Mean F1 Score: {results_df['f1'].mean():.4f} ± {results_df['f1'].std():.4f}")

# 95% confidence interval
f1_mean = results_df['f1'].mean()
f1_std = results_df['f1'].std()
f1_ci_lower = f1_mean - 1.96 * f1_std / np.sqrt(N_FOLDS)
f1_ci_upper = f1_mean + 1.96 * f1_std / np.sqrt(N_FOLDS)
print(f"\n95% Confidence Interval: [{f1_ci_lower:.4f}, {f1_ci_upper:.4f}]")

# Plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(results_df['fold'], results_df['accuracy'])
plt.axhline(y=results_df['accuracy'].mean(), color='r', linestyle='--', label='Mean')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.title('Accuracy per Fold')
plt.legend()
plt.ylim([0.7, 1.0])

plt.subplot(1, 2, 2)
plt.bar(results_df['fold'], results_df['f1'])
plt.axhline(y=results_df['f1'].mean(), color='r', linestyle='--', label='Mean')
plt.xlabel('Fold')
plt.ylabel('F1 Score')
plt.title('F1 Score per Fold')
plt.legend()
plt.ylim([0.7, 1.0])

plt.tight_layout()
plt.show()

## 11. Ensemble Predictions with Cascading Classifier

Apply cascading classifier first, then use ensemble for uncertain cases.

In [ ]:
print("="*80)
print("GENERATING PREDICTIONS ON DEV SET")
print("="*80)

# Initialize predictions array
n_dev = len(dev_df)
final_predictions = np.full(n_dev, -1)  # -1 means not yet predicted
final_confidences = np.zeros(n_dev)
prediction_source = [''] * n_dev  # Track prediction source

# Step 1: Apply cascading classifier (if enabled and toxic_ratio available)
if USE_CASCADING and TOXIC_RATIO_COL in dev_df.columns:
    print("\n📊 Step 1: Applying Cascading Classifier...")
    
    toxic_ratios = dev_df[TOXIC_RATIO_COL].values
    
    # High toxic -> TOXIC (class 1)
    high_toxic_mask = toxic_ratios > TOXIC_THRESHOLD
    final_predictions[high_toxic_mask] = 1
    final_confidences[high_toxic_mask] = 0.9
    for i in np.where(high_toxic_mask)[0]:
        prediction_source[i] = 'rule_toxic'
    
    # Very clean -> CLEAN (class 0)
    very_clean_mask = toxic_ratios < CLEAN_THRESHOLD
    final_predictions[very_clean_mask] = 0
    final_confidences[very_clean_mask] = 0.85
    for i in np.where(very_clean_mask)[0]:
        prediction_source[i] = 'rule_clean'
    
    n_rule_based = high_toxic_mask.sum() + very_clean_mask.sum()
    print(f"  Rule-based predictions: {n_rule_based} ({100*n_rule_based/n_dev:.1f}%)")
    print(f"    - High toxic -> TOXIC: {high_toxic_mask.sum()}")
    print(f"    - Very clean -> CLEAN: {very_clean_mask.sum()}")
else:
    print("\n⚠️ Cascading classifier disabled or toxic_ratio not available")
    n_rule_based = 0

# Step 2: Get indices that need model prediction
uncertain_mask = final_predictions == -1
uncertain_indices = np.where(uncertain_mask)[0]
n_uncertain = len(uncertain_indices)

print(f"\n🤖 Step 2: Model prediction needed for {n_uncertain} samples ({100*n_uncertain/n_dev:.1f}%)")

In [ ]:
# Step 3: Get ensemble predictions for uncertain samples
if n_uncertain > 0:
    print("\n🔄 Getting ensemble predictions for uncertain samples...")
    
    # Create dataset for uncertain samples only
    uncertain_df = dev_df.iloc[uncertain_indices].reset_index(drop=True)
    uncertain_dataset = Dataset.from_pandas(uncertain_df[[TEXT_COL]])
    uncertain_dataset = uncertain_dataset.map(tokenize_function, batched=True)
    uncertain_dataset.set_format('torch', columns=['input_ids', 'attention_mask'])
    
    # Collect predictions from all models
    all_model_probs = []
    
    for fold, model_path in enumerate(model_paths, 1):
        print(f"  Loading Fold {fold} model...")
        
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        model.eval()
        model.to('cuda' if torch.cuda.is_available() else 'cpu')
        
        temp_trainer = Trainer(
            model=model,
            args=TrainingArguments(
                output_dir='./temp',
                per_device_eval_batch_size=32,
                fp16=torch.cuda.is_available(),
                report_to='none'
            ),
            data_collator=data_collator
        )
        
        predictions = temp_trainer.predict(uncertain_dataset)
        probs = torch.softmax(torch.tensor(predictions.predictions), dim=1).numpy()
        all_model_probs.append(probs)
        
        del model, temp_trainer, predictions
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
    
    # Ensemble: average probabilities
    all_model_probs = np.array(all_model_probs)
    ensemble_probs = all_model_probs.mean(axis=0)
    ensemble_preds = np.argmax(ensemble_probs, axis=1)
    ensemble_conf = ensemble_probs.max(axis=1)
    
    # Fill in uncertain predictions
    for i, (orig_idx, pred, conf) in enumerate(zip(uncertain_indices, ensemble_preds, ensemble_conf)):
        final_predictions[orig_idx] = pred
        final_confidences[orig_idx] = conf
        prediction_source[orig_idx] = 'ensemble'
    
    print(f"\n✓ Ensemble predictions complete for {n_uncertain} samples")

print(f"\n✓ All predictions complete!")

## 12. Prediction Summary

In [ ]:
print("="*80)
print("PREDICTION SUMMARY")
print("="*80)

# Prediction distribution
print(f"\nPrediction Distribution:")
unique, counts = np.unique(final_predictions, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  Class {label}: {count} ({100*count/len(final_predictions):.1f}%)")

# Source distribution
print(f"\nPrediction Source:")
source_counts = pd.Series(prediction_source).value_counts()
for source, count in source_counts.items():
    print(f"  {source}: {count} ({100*count/len(prediction_source):.1f}%)")

# Confidence statistics
print(f"\nConfidence Statistics:")
print(f"  Mean: {final_confidences.mean():.3f}")
print(f"  Min: {final_confidences.min():.3f}")
print(f"  Max: {final_confidences.max():.3f}")

# High confidence count
high_conf = (final_confidences > 0.9).sum()
print(f"  High confidence (>0.9): {high_conf} ({100*high_conf/len(final_confidences):.1f}%)")

## 13. Create Submission File

In [ ]:
# Create submission dataframe
submission_df = pd.DataFrame({
    'id': dev_df['id'],
    'polarization': final_predictions.astype(int)
})

# Save to CSV
output_file = 'pred_arb_pipeline_ensemble.csv'
submission_df.to_csv(output_file, index=False)

print(f"✓ Submission file created: {output_file}")
print(f"\nFile preview:")
print(submission_df.head(10))
print(f"\nTotal predictions: {len(submission_df)}")

In [ ]:
# Show class balance
print("Predicted class distribution:")
print(submission_df['polarization'].value_counts())

## 14. Verification

In [ ]:
print("="*80)
print("VERIFYING SUBMISSION FILE")
print("="*80)

verify_df = pd.read_csv(output_file)

# Check columns
expected_columns = ['id', 'polarization']
if list(verify_df.columns) == expected_columns:
    print("✓ Columns are correct")
else:
    print(f"✗ Column mismatch!")

# Check for missing values
if verify_df.isnull().sum().sum() == 0:
    print("✓ No missing values")
else:
    print(f"✗ Missing values found")

# Check values
if set(verify_df['polarization'].unique()).issubset({0, 1}):
    print(f"✓ Polarization values are valid: {sorted(verify_df['polarization'].unique())}")
else:
    print(f"✗ Invalid values")

# Check count
if len(verify_df) == len(dev_df):
    print(f"✓ Prediction count matches: {len(verify_df)}")
else:
    print(f"✗ Count mismatch")

print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)
print(f"\n📄 File: {output_file}")
print(f"📊 Total predictions: {len(verify_df)}")
print(f"\n🎯 Training Performance ({N_FOLDS}-Fold CV):")
print(f"   Mean F1 Score: {results_df['f1'].mean():.4f} ± {results_df['f1'].std():.4f}")
if USE_CASCADING and TOXIC_RATIO_COL in dev_df.columns:
    print(f"\n⚡ Cascading Classifier:")
    print(f"   Rule-based predictions: {n_rule_based} ({100*n_rule_based/n_dev:.1f}%)")
    print(f"   Model predictions: {n_uncertain} ({100*n_uncertain/n_dev:.1f}%)")

## Summary

### Pipeline Integration
- **Input:** Preprocessed data from preprocessing pipeline
- **Text Column:** `text_normalized` (dialect-normalized text)
- **Features Used:** `toxic_ratio` for cascading classifier

### Cascading Classifier
- **Toxic threshold:** > 0.15 → Predict TOXIC (confidence 0.9)
- **Clean threshold:** < 0.02 → Predict CLEAN (confidence 0.85)
- **Uncertain:** Falls back to ensemble model prediction

### Ensemble
- **Models:** 8 MARBERT v2 models (one per fold)
- **Method:** Soft voting (averaged probabilities)
- **Memory efficient:** One model loaded at a time

### Output
- **File:** `pred_arb_pipeline_ensemble.csv`
- **Format:** CSV with `id` and `polarization` columns